# 카카오 이미지 검색결과 수집하기

- 패션 잡지에 넣을 `봄`을 주제로 한 `옷차림`을 알아보고자 한다. `해당 주제로 검색된 이미지를 수집`하여라
## 1. 필요한 모듈 참조

In [1]:
# -> 웹 상의 데이터를 요청하는 기능
import requests
    # request로 갖고 온 데이터값이 JSON형태이면 dictionary로 변환하면 된다
    # request로 갖고 온 데이터값이 HTML + CSS 이면 BeautifulSoup라는 클래스를 통해 HTML + CSS 안에 있는 내용들을 추출해준다
# -> JSON을 딕셔너리로 변환하는 기능(파이썬 내장모듈이기 때문에 설치 불필요)
import json
# DataFrame - 딕셔너리를 포함하는 리스트를 표 형태로 만들어줄 때 씀; 엑셀 시트 같이 생긴 자료구조
from pandas import DataFrame

import os

## 2. url 객체 생성 (`url 기본 형식 + 요청 인터페이스`)
- Kakao Developers에서는 발급받아야 하는 API key가 있다.
- 원하는 데이터에 따라 접속하는 방법을 설명하는 메뉴얼을 확인할 수 있다.
    - 메뉴얼 확인하기: Kakao Developers > 로그인 > 위 메뉴에서 `문서` > 원하는 데이터 수집 관련 내용 확인할 수 있음
    - 우리는 `다음에서 카카오 이미지를 수집`할 것이기에: `메뉴` > `Daum 검색` > `REST API` > `이미지 검색하기` 
### 0) 작업 준비
#### i. REST API 키 받기
- Kakao Developers에 들어가서 로그인 > `내 애플리케이션` > `전체 애플리케이션`에서 생성한 `애플리케이션` > `REST API 키` 복사: `f903e6bb99d409d576298b6e08527c66`
#### ii. 책 검색하기 위한 필수/원하는 파라미터들 확인
- `메뉴` > `Daum 검색` > `REST API` > `이미지 검색하기` 
    - 요청: 우리가 요청하는 것
    - 응답: 우리가 요청했을 때 받는 데이터
        - Meta: 응답 관련 정보, 즉 원 데이터를 설명하는 부가 정보
        - Documents: 응답 결과 (이 예제에서 우리가 얻고자 하는 것)
- url 및 필수 파라미터
    - URL: `기본 정보` > URL 복사: `https://dapi.kakao.com/v2/search/image`
    - Authorization: `요청` > `헤더` > `Authorization` 복사: `Authorization: KakaoAK ${REST_API_KEY}`
    - query (String; 검색을 원하는 질의어)
- 추가(원하는) 파라미터:
    - page (Integer; 결과 페이지 번호, 1~50 사이의 값, 기본 값 1)
    - size (Integer; 한 페이지에 보여질 문서 수, 1~50 사이의 값, 기본 값 10)
### 1) `url 기본 형식` 객체 생성
- url 상 `get 파라미터`의 경우 `?` 다음에 넣고 여러개일 경우 중간에 `&`를 넣는다
- url에 넣을 파라미터: 
    - query
    - page
    - size
- 문법 예시:
    - 'https://dapi.kakao ... /image`?`query={질의어}`&`page={숫자}&size={숫자}'

In [2]:
# 접속할 URL
urlFmt = "https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}"

### 2) `요청변수 및 발급받은 키` 객체 생성

In [3]:
# 요청변수 (파라미터)
query = "봄 패션"
page = 1
size = 80
key = "b636ff83df45f778874b079dcb979102"

## 3. 접속 객체 생성 및 접속 정보 설정

In [4]:
# 접속객체 생성
session = requests.Session()

# 접속객체에 부가정보(header) 삽입하기
session.headers.update({
    "Authorization": "KakaoAK %s" % key
})

## 4. 반복처리 하며...
### `접속할 URL 생성` + `원하는 웹페이지에 접속` + `json을 딕셔너리로 변환` + `이미지 수집`

In [5]:
# 다음 페이지가 존재하는지 여부 --> False인 동안은 while문 반복
isEnd = False

# 1페이지부터 시작
page = 1

# 저장된 이미지 카운트
count = 0

while (not isEnd):
    #--------------------------------------------------------------
    # 이전 예제 코드 시작
    #--------------------------------------------------------------
    # 접속할 URL
    url = urlFmt.format(query=query, page=page, size=size)
    url

    # 생성한 접속객체를 활용하여 API에 접속
    r = session.get(url)

    # 접속에 실패한 경우
    if r.status_code != 200:
        # 에러코드와 에러메시지 출력
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        # 에러를 강제로 생성시킴
        raise Exception(msg)

    # 인코딩 형식 지정
    r.encoding = "utf-8"
    # 가져온 문자열을 딕셔너리로 변환
    data = json.loads(r.text)
    #data

    #--------------------------------------------------------------
    # 이전 예제 코드 끝
    #--------------------------------------------------------------

    # 검색 결과를 갖고 있는 딕셔너리를 포함하는 리스트 추출
    searchList = data['documents']

    # 증감식에 해당하는 isEnd변수 추출
    isEnd = bool(data['meta']['is_end'])
    page += 1
    print("isEnd=", isEnd, " / 다음페이지=", page)

    # 이미지가 저장될 폴더 생성
    if os.path.exists(query) == False:
        os.mkdir(query)

    # 이미지 저장
    for i, v in enumerate(searchList):
        # 숫자 카운트
        count += 1

        # 이미지 URL 추출
        docUrl = v['doc_url']
        imageUrl = v['image_url']
        print(imageUrl)
        
        # 접속객체에 부가정보(header) 삽입하기
        session.headers.update({
            "Authorization": "KakaoAK %s" % key,
            "Referer": docUrl,
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
        })

        # 텍스트 이외의 파일 형식을 가져오기 위해서는 stream 파라미터 추가
        try:
            r = session.get(imageUrl, stream=True)
        except:
            continue

        # 다운로드에 실패한 경우 다음 이미지로 이동
        if r.status_code != 200:
            continue

        fileName = "%s/%05d.png" % (query, count)

        with open(fileName , 'wb') as f:
            # 통신으로 가져온 내용의 바이너리(기계어)를 직접 추출
            f.write(r.raw.read())
            print(fileName, "(이)가 저장되었습니다.")

print("종료")

isEnd= False  / 다음페이지= 2
http://cfile261.uf.daum.net/image/144B9C544DB691270CA6C9
봄 패션/00001.png (이)가 저장되었습니다.
https://img.insight.co.kr/static/2023/01/30/700/img_20230130115256_gyw8w1j3.webp
봄 패션/00002.png (이)가 저장되었습니다.
https://postfiles.pstatic.net/20140307_148/heeyoo7379_1394193363157tdyHs_JPEG/%BA%BD_%C6%D0%BC%C7%2C_%B2%C9_%B0%A1%B5%F0%B0%C7%2C_%B2%C9_%C6%D0%BC%C7.jpg?type=w1
봄 패션/00003.png (이)가 저장되었습니다.
https://t1.daumcdn.net/news/201806/25/joongang/20180625000426823akze.jpg
봄 패션/00004.png (이)가 저장되었습니다.
https://postfiles.pstatic.net/20150317_164/wjdwktldldl_1426567474758tagoH_JPEG/2015-03-17_13-42-46.jpg?type=w2
봄 패션/00005.png (이)가 저장되었습니다.
https://postfiles.pstatic.net/20140322_6/jkkk0409_13954651210977UIm6_JPEG/img_39811.jpg?type=w2
봄 패션/00006.png (이)가 저장되었습니다.
http://cafeptthumb3.phinf.naver.net/20130117_12/chj7555_1358400393201hsv5S_JPEG/%C4%B8%C3%B319.JPG?type=w740
https://t1.daumcdn.net/cfile/tistory/1946D8445147D58007
봄 패션/00008.png (이)가 저장되었습니다.
https://dthumb-phinf.pstati